In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import glob
import numpy as np
import matplotlib.pyplot as pl
import ipdb
import platform
import h5py
import scipy.io
opj = os.path.join
deb = ipdb.set_trace

# MRI imports
# -----------
import nibabel as nb
import cortex

# Bokeh imports
# -------------
from bokeh.io import output_notebook, show, save, output_file, export_png, export_svgs
from bokeh.layouts import row, column, gridplot

# Popeye imports
# --------------
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus
import popeye.css as css
import popeye.og as og

# Get inputs
# ----------
subject = '999999'
fit_model = 'gauss'
draw_hemi = 1

In [2]:
# Define analysis parameters
# --------------------------
with open('../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
    
# Define cluster/server specific parameters
# -----------------------------------------
if 'aeneas' in platform.uname()[1]:
    base_dir = analysis_info['aeneas_base_folder'] 
elif 'local' in platform.uname()[1]:
    base_dir = analysis_info['local_base_folder'] 
    
h5_dir = opj(base_dir,'pp_data',subject,fit_model,'h5')
base_file_name = 'tfMRI_RETBAR1_7T_AP_Atlas_MSMAll_hp2000_clean.dtseries'
roi_masks_dir = opj(base_dir,'pp_data',subject,fit_model,'roi_masks')
sign_idx, rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
            non_lin_idx, amp_idx, baseline_idx, cov_idx, x_idx, y_idx = 0,1,2,3,4,5,6,7,8,9,10,11
    
# Create stimulus design and define model
visual_dm_file = scipy.io.loadmat(opj(base_dir,'raw_data','retinotopysmall5.mat'))
visual_dm = visual_dm_file['stim']

stimulus = VisualStimulus(  stim_arr = visual_dm,
                            viewing_distance = analysis_info["screen_distance"],
                            screen_width = analysis_info["screen_width"],
                            scale_factor = 1/10.0,
                            tr_length = analysis_info["TR"],
                            dtype = np.short)


if fit_model == 'gauss':
    fit_func = og.GaussianFit
    num_est = 6
    model_func = og.GaussianModel(  stimulus = stimulus,
                                    hrf_model = utils.spm_hrf)
    step_r2 = [0,50,100] # [low or high]
    step_params = [0,50,100] # [low or high]
    list_params = ['ecc','amp','cov']
    num_plots = len(list_params)*len(step_params)*(len(step_r2)-1)
    
elif fit_model == 'css':
    fit_func = css.CompressiveSpatialSummationFit
    num_est = 7
    model_func = css.CompressiveSpatialSummationModel(  stimulus = stimulus,
                                                        hrf_model = utils.spm_hrf)
    step_r2 = [0,50,100] # [low or high]
    step_params = [0,50,100] # [low or high]
    list_params = ['ecc','amp','cov','non_lin']
    num_plots = len(list_params)*len(step_params)*(len(step_r2)-1)
    
model_func.hrf_delay = 0


In [83]:
# Draw main analysis figure
# -------------------------
print('creating bokeh time course plots')

# Initialize data dictionary that will save all data arrays

for roi_num, roi in enumerate(["V1"]):
    roi_text = roi

    for mask_dir in ['pos']:#['pos','neg']:
        deriv_mat_hemi = []
        tc_mat_hemi = []
        val_hemi = 0
        for hemi in ['L']:#['L', 'R','LR']:
            # load data
            if hemi == 'LR':
                deriv_mat = np.row_stack((deriv_mat_hemi[0],deriv_mat_hemi[1]))
                draw = True
            else:
                # load roi masks
                if hemi == 'L' or hemi == 'R':
                    mask_file = opj(roi_masks_dir,"masks_{hemi}.func.gii".format(hemi = hemi))
                    gii_in_mask = nb.load(mask_file)
                    mask_mat = np.array([gii_in_mask.darrays[i].data for i in range(len(gii_in_mask.darrays))])

                    # load time course of original data
                    orig_data_file  =  sorted(glob.glob(opj(base_dir,'raw_data',subject,'*RETBAR1_7T*%s.func_bla_psc_av.gii'% hemi)))
                    orig_data_file_load = nb.load(orig_data_file[0])
                    orig_data = []
                    orig_data.append(np.array([orig_data_file_load.darrays[i].data for i in range(len(orig_data_file_load.darrays))]))
                    orig_data = np.vstack(orig_data)
                
                # select corresponding time course
                mask_mat_roi = mask_mat[roi_num,:]
                mask_mat_roi = np.round(mask_mat_roi)
                tc_mat = orig_data[:, mask_mat_roi==1].T
                tc_mat_hemi.append(tc_mat)
                
                # load derivatives
                if hemi == 'L': val_hemi = 1
                elif hemi == 'R': val_hemi = 2
                folder_alias = '{hemi}_{mask_dir}'.format(hemi = hemi,mask_dir = mask_dir)
                h5_file = h5py.File(opj(h5_dir,'{roi}.h5'.format(roi = roi_text)), "r")
                in_file = opj("prf_deriv_{hemi}_{mask_dir}".format(hemi = hemi, mask_dir = mask_dir))
                deriv_mat = h5_file['{folder_alias}/{in_file}'.format(folder_alias=folder_alias,in_file=in_file)]
                deriv_mat = np.vstack((deriv_mat,val_hemi*np.ones((1,deriv_mat.shape[1]))))
                deriv_mat = deriv_mat[:,:].T
                deriv_mat_hemi.append(deriv_mat)

                deriv_mat[:,rsq_idx]
#                 # create model tc
#                 if draw_hemi == 1:
#                     draw = True
#                 elif draw_hemi == 0:
#                     draw = False                

creating bokeh time course plots


In [232]:
# get index matrices
prct_r2 = np.nanpercentile(a = deriv_mat[:,rsq_idx],q = step_r2)
idx_r2 = []
for r2_step in np.arange(0,len(step_r2)-1,1):
    idx_r2.append(np.logical_and(deriv_mat[:,rsq_idx]>prct_r2[r2_step],deriv_mat[:,rsq_idx]<=prct_r2[r2_step+1]))

for param in list_params:
    exec('prct_{param} = np.nanpercentile(a = deriv_mat[:,{param}_idx],q = step_params)'.format(param = param))
    exec('idx_{param} = []'.format(param = param))

    for param_step in np.arange(0,len(step_params)-1,1):
        exec('idx_{param}.append(np.logical_and(deriv_mat[:,{param}_idx]>prct_{param}[param_step],deriv_mat[:,{param}_idx]<=prct_{param}[param_step+1]))'.format(param = param))

# get vertex number to draw
for param in list_params:
    exec('num_{param} = []'.format(param = param))
    exec('num_{param} = []'.format(param = param))
    for r2_step in np.arange(0,len(step_r2)-1,1):
        for param_step in np.arange(0,len(step_params)-1,1):
            exec('mat = np.where(np.logical_and(idx_r2[r2_step],idx_{param}[param_step]))'.format(param = param))
            exec('num_{param}.append(mat[0][np.random.choice(len(mat[0]))])'.format(param = param))
                    

In [233]:
def get_prediction(fit_model,num_vertex):
    # get data time course
    tc_data_mat = tc_mat[num_vertex,:]
    
    # get model time course
    if fit_model == 'gauss':
        tc_model_mat = model_func.generate_prediction( 
                                            x = deriv_mat[num_vertex,x_idx], 
                                            y = deriv_mat[num_vertex,y_idx], 
                                            sigma = deriv_mat[num_vertex,size_idx],
                                            beta = deriv_mat[num_vertex,amp_idx], 
                                            baseline = deriv_mat[num_vertex,baseline_idx])
    elif fit_model == 'css':
        tc_model_mat = model_func.generate_prediction( 
                                            x = deriv_mat[num_vertex,x_idx], 
                                            y = deriv_mat[num_vertex,y_idx], 
                                            sigma = deriv_mat[num_vertex,size_idx],
                                            beta = deriv_mat[num_vertex,amp_idx], 
                                            n = deriv_mat[num_vertex,non_lin_idx], 
                                            baseline = deriv_mat[num_vertex,baseline_idx])
    
    deriv_model_mat = deriv_mat[num_vertex,:]

    return (tc_data_mat, tc_model_mat,deriv_model_mat)

In [235]:
from __future__ import division, print_function
# Bokeh import
# ------------
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models.annotations import Span, Label
from bokeh.layouts import row, column, gridplot
from bokeh.models import BoxZoomTool, BoxSelectTool, Spacer, WheelZoomTool, PanTool, ResetTool
from bokeh.models.glyphs import Text
from bokeh.models.mappers import LinearColorMapper
from bokeh.io import output_notebook, show,save, output_file, export_png, export_svgs
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
output_notebook()

# Other imports
# -------------
import numpy as np
import cortex
import matplotlib.colors as colors
import scipy as sy
import numpy as np
import ipdb
deb = ipdb.set_trace

from popeye.spinach import generate_og_receptive_fields
from IPython import embed as shell

p_width = 500
p_height = 500
min_border_large = 10

x_range_map = (-12,12)
y_range_map = (-12,12)
x_label_map = 'Horizontal coord. (dva)'
y_label_map = 'Vertical coord. (dva)'
x_tick_map = 4
y_tick_map = 4

x_range_tc = (0,300)
x_label_tc = 'Time (s)'
y_label_tc = 'BOLD (%)'
x_tick_tc = 50


stim_radius = analysis_info['stim_radius']
tr_dur = analysis_info['TR']
stim_color = tuple([250,250,250])
bg_color = tuple([229,229,229])
model_line_color = tuple([254,51,10])
model_fill_color = tuple([254,51,10])

params = 'ecc'
val_r2 = 'High'

# Time course - low parameter
# ---------------------------
# get model and data time course
exec('tc_data_mat,tc_model_mat,deriv_model_mat = get_prediction(fit_model = fit_model,num_vertex = num_{params}[1])'.format(params = params))
y_range_tc = (np.round(np.nanmin(tc_data_mat)*1.5,0),np.round(np.nanmax(tc_data_mat)*1.5,0))

# get data
x_data = np.arange(1,tc_data_mat.shape[0]+1,1)*tr_dur
y_data = tc_data_mat
x_model = np.arange(1,tc_model_mat.shape[0]+1,1)*tr_dur
y_model = tc_model_mat

high_param_tc_data_source = { 'x_data':x_data,
                              'y_data':y_data,
                              'x_model':x_model,
                              'y_model':y_model}
high_param_tc_source = ColumnDataSource(data = high_param_tc_data_source)

high_param_tc_fig              =   figure(
                                        plot_width          =   p_width,
                                        plot_height         =   int(p_height*0.42),
                                        x_range             =   x_range_tc,
                                        y_range             =   y_range_tc,
                                        min_border_left     =   min_border_large,
                                        min_border_right    =   min_border_large,
                                        min_border_bottom   =   min_border_large,
                                        min_border_top      =   min_border_large,
                                        toolbar_location    =   None,
                                        tools               =   "")
high_param_tc_fig.xaxis.axis_label       =   ''
high_param_tc_fig.yaxis.axis_label       =   y_label_tc
high_param_tc_fig.grid.grid_line_color   =   None
high_param_tc_fig.axis.minor_tick_in     =   False
high_param_tc_fig.axis.minor_tick_out    =   False
high_param_tc_fig.axis.major_tick_in     =   False
high_param_tc_fig.outline_line_alpha     =   0
high_param_tc_fig.xaxis.ticker           =   np.arange(x_range_tc[0],x_range_tc[1] + x_tick_tc, x_tick_tc)
high_param_tc_fig.background_fill_color  =   bg_color
high_param_tc_fig.axis.axis_label_standoff = 10
high_param_tc_fig.axis.axis_label_text_font_style = 'normal'
high_param_tc_fig.xaxis.major_label_text_font_size = '0pt'

# span
high_param_tc_fig.add_layout(Span(location = 0, dimension = 'width', line_alpha = 0.5, line_color = 'black', line_width = 1, line_dash = 'dashed'))

# plot data
high_param_tc_fig.circle(x = 'x_data', y = 'y_data',fill_color = 'black',line_color = 'black',line_width = 1,size = 2,source=high_param_tc_source, legend = "data")

# plot data model
high_param_tc_plot_model = high_param_tc_fig.line(x = 'x_model', y = 'y_model', line_width = 2, line_color = model_line_color, source = high_param_tc_source, legend = "model")

# data hover
high_param_tc_fig_tooltips = [  ('data: ',  ' @x_data{0} s, @y_data{0.0} %'),
                                ('model: ',  '@x_model{0} s, @y_model{0.0} %')
                            ]                               # coverage
high_param_tc_fig_hover = HoverTool(tooltips = high_param_tc_fig_tooltips,
                                    mode = 'vline',
                                    renderers = [high_param_tc_plot_model])
high_param_tc_fig.add_tools(high_param_tc_fig_hover)

# legend
high_param_tc_fig.legend.location = "top_right"
high_param_tc_fig.legend.click_policy="hide"
high_param_tc_fig.legend.background_fill_alpha = 0
high_param_tc_fig.legend.label_text_font = '8pt'
high_param_tc_fig.legend.margin = 5
high_param_tc_fig.legend.padding = 5
high_param_tc_fig.legend.spacing = -5
high_param_tc_fig.legend.glyph_width = 10
high_param_tc_fig.legend.glyph_height = 10
high_param_tc_fig.legend.label_text_baseline = 'bottom'

# text
x_text = x_range_tc[0] + (x_range_tc[1]-x_range_tc[0])*0.05
y_text1 = y_range_tc[1] - (y_range_tc[1]-y_range_tc[0])*0.11
y_text2 = y_range_tc[1] - (y_range_tc[1]-y_range_tc[0])*0.19
text1 = '{val_r2} R2 | High {params}'.format(val_r2 = val_r2, params = params)
if fit_model == 'gauss':
    text2 = '[R2: {:1.1f}; ecc: {:1.1f} dva; size: {:1.1f} dva; cov = {:1.0f} %]'.format(
                                                                                    deriv_model_mat[rsq_idx],
                                                                                    deriv_model_mat[ecc_idx],
                                                                                    deriv_model_mat[size_idx],
                                                                                    deriv_model_mat[cov_idx]*100,
                                                                                     )
elif fit_model == 'css':
    text2 = '[R2: {:1.1f}; ecc: {:1.1f} dva; size: {:1.1f} dva; n: {:1.1f}; cov: {:1.0f} %]'.format(deriv_model_mat[ecc_idx],
                                                                                     deriv_model_mat[rsq_idx],
                                                                                     deriv_model_mat[size_idx],
                                                                                     deriv_model_mat[non_lin_idx],
                                                                                     deriv_model_mat[cov_idx]*100,
                                                                                     params = params,)
high_param_tc_fig.text(x=x_text,y=y_text1,text = [text1],text_font_size = '8pt',text_font_style = 'bold')
high_param_tc_fig.text(x=x_text,y=y_text2,text = [text2],text_font_size = '8pt')

# pRF map - low parameter
# -----------------------
high_param_map_fig              =   figure(
                                        plot_width          =   int(p_height/2),
                                        plot_height         =   int(p_height*0.42),
                                        x_range             =   x_range_map,
                                        y_range             =   y_range_map,
                                        min_border_left     =   min_border_large,
                                        min_border_right    =   min_border_large,
                                        min_border_bottom   =   min_border_large,
                                        min_border_top      =   min_border_large,
                                        toolbar_location    =   None,
                                        tools               =   "")

high_param_map_fig.xaxis.axis_label       =   ''
high_param_map_fig.yaxis.axis_label       =   y_label_map
high_param_map_fig.grid.grid_line_color   =   None
high_param_map_fig.axis.minor_tick_in     =   False
high_param_map_fig.axis.minor_tick_out    =   False
high_param_map_fig.axis.major_tick_in     =   False
high_param_map_fig.outline_line_alpha     =   0
high_param_map_fig.yaxis.ticker           =   np.arange(y_range_map[0],y_range_map[1] + y_tick_map, y_tick_map)
high_param_map_fig.xaxis.ticker           =   np.arange(x_range_map[0],x_range_map[1] + x_tick_map, x_tick_map)
high_param_map_fig.background_fill_color  =   bg_color
high_param_map_fig.axis.axis_label_standoff = 10
high_param_map_fig.axis.axis_label_text_font_style = 'normal'
high_param_map_fig.xaxis.major_label_text_font_size = '0pt'

# stimulus circle
high_param_map_fig.circle(x = 0, y = 0, radius = stim_radius, color = stim_color)

# spans
high_param_map_fig.add_layout(Span(location = 0, dimension = 'width', line_alpha = 0.5, line_color = 'black', line_width = 1, line_dash = 'dashed'))
high_param_map_fig.add_layout(Span(location = 0, dimension = 'height', line_alpha = 0.5, line_color = 'black', line_width = 1, line_dash = 'dashed'))

# plot rf
high_param_map_fig.circle(  x                   =   deriv_model_mat[x_idx],
                            y                   =   deriv_model_mat[y_idx],
                            radius              =   deriv_model_mat[sign_idx],
                            fill_color          =   model_fill_color,
                            line_color          =   'black',
                            fill_alpha          =   1,
                            line_alpha          =   1)

# Time course - low parameter
# ----------------------------
# get model and data time course
exec('tc_data_mat,tc_model_mat,deriv_model_mat = get_prediction(fit_model = fit_model,num_vertex = num_{params}[0])'.format(params = params))
y_range_tc = (np.round(np.nanmin(tc_data_mat)*1.5,0),np.round(np.nanmax(tc_data_mat)*1.5,0))

# get data
x_data = np.arange(1,tc_data_mat.shape[0]+1,1)*tr_dur
y_data = tc_data_mat
x_model = np.arange(1,tc_model_mat.shape[0]+1,1)*tr_dur
y_model = tc_model_mat

low_param_tc_data_source = {  'x_data':x_data,
                              'y_data':y_data,
                              'x_model':x_model,
                              'y_model':y_model}
low_param_tc_source = ColumnDataSource(data = low_param_tc_data_source)

low_param_tc_fig              =   figure(
                                        plot_width          =   p_width,
                                        plot_height         =   int(p_height/2),
                                        x_range             =   x_range_tc,
                                        y_range             =   y_range_tc,
                                        min_border_left     =   min_border_large,
                                        min_border_right    =   min_border_large,
                                        min_border_bottom   =   min_border_large,
                                        min_border_top      =   min_border_large,
                                        toolbar_location    =   None,
                                        tools               =   "")

low_param_tc_fig.xaxis.axis_label       =   x_label_tc
low_param_tc_fig.yaxis.axis_label       =   y_label_tc
low_param_tc_fig.grid.grid_line_color   =   None
low_param_tc_fig.axis.minor_tick_in     =   False
low_param_tc_fig.axis.minor_tick_out    =   False
low_param_tc_fig.axis.major_tick_in     =   False
low_param_tc_fig.outline_line_alpha     =   0
low_param_tc_fig.xaxis.ticker           =   np.arange(x_range_tc[0],x_range_tc[1] + x_tick_tc, x_tick_tc)
low_param_tc_fig.background_fill_color  =   bg_color
low_param_tc_fig.axis.axis_label_standoff = 10
low_param_tc_fig.axis.axis_label_text_font_style = 'normal'
low_param_tc_fig.add_layout(Span(location = 0, dimension = 'width', line_alpha = 0.5, line_color = 'black', line_width = 1, line_dash = 'dashed'))

# plot data
low_param_tc_fig.circle(x = 'x_data', y = 'y_data', fill_color = 'black', line_color = 'black', line_width = 1, size = 2, source = low_param_tc_source, legend="data" )

# plot data model
low_param_tc_plot_model = low_param_tc_fig.line(x = 'x_model', y = 'y_model', line_width = 2, line_color = model_line_color, source = low_param_tc_source, legend="model")


# data hover
low_param_tc_fig_tooltips = [   ('data: ',  ' @x_data{0} s, @y_data{0.0} %'),
                                ('model: ',  '@x_model{0} s, @y_model{0.0} %')
                            ]                               # coverage
low_param_tc_fig_hover = HoverTool(tooltips = low_param_tc_fig_tooltips,
                                    mode = 'vline',
                                    renderers = [low_param_tc_plot_model])
low_param_tc_fig.add_tools(low_param_tc_fig_hover)

# legend
low_param_tc_fig.legend.location = "top_right"
low_param_tc_fig.legend.click_policy="hide"
low_param_tc_fig.legend.background_fill_alpha = 0
low_param_tc_fig.legend.label_text_font = '8pt'
low_param_tc_fig.legend.margin = 5
low_param_tc_fig.legend.padding = 5
low_param_tc_fig.legend.spacing = -5
low_param_tc_fig.legend.glyph_width = 10
low_param_tc_fig.legend.glyph_height = 10
low_param_tc_fig.legend.label_text_baseline = 'bottom'

# text
x_text = x_range_tc[0] + (x_range_tc[1]-x_range_tc[0])*0.05
y_text1 = y_range_tc[1] - (y_range_tc[1]-y_range_tc[0])*0.11
y_text2 = y_range_tc[1] - (y_range_tc[1]-y_range_tc[0])*0.19
text1 = '{val_r2} R2 | Low {params}'.format(val_r2 = val_r2, params = params)
if fit_model == 'gauss':
    text2 = '[R2: {:1.1f}; ecc: {:1.1f} dva; size: {:1.1f} dva; cov = {:1.0f} %]'.format(
                                                                                    deriv_model_mat[rsq_idx],
                                                                                    deriv_model_mat[ecc_idx],
                                                                                    deriv_model_mat[size_idx],
                                                                                    deriv_model_mat[cov_idx]*100,
                                                                                     )
elif fit_model == 'css':
    text2 = '[R2: {:1.1f}; ecc: {:1.1f} dva; size: {:1.1f} dva; n: {:1.1f}; cov: {:1.0f} %]'.format(deriv_model_mat[ecc_idx],
                                                                                     deriv_model_mat[rsq_idx],
                                                                                     deriv_model_mat[size_idx],
                                                                                     deriv_model_mat[non_lin_idx],
                                                                                     deriv_model_mat[cov_idx]*100,
                                                                                     params = params,)
low_param_tc_fig.text(x=x_text,y=y_text1,text = [text1],text_font_size = '8pt',text_font_style = 'bold')
low_param_tc_fig.text(x=x_text,y=y_text2,text = [text2],text_font_size = '8pt')


# pRF map - high parameter
# -----------------------
low_param_map_fig              =   figure(                                                                  # create a figure in bokeh
                                        plot_width          =   int(p_height/2),                            # define figure width in pixel
                                        plot_height         =   int(p_height/2),                            # define figure height in pixel
                                        x_range             =   x_range_map,                               # define x limits
                                        y_range             =   y_range_map,                               # define y limits
                                        min_border_left     =   min_border_large,                      # define left border size
                                        min_border_right    =   min_border_large,                      # define right border size
                                        min_border_bottom   =   min_border_large,                      # define bottom border space
                                        min_border_top      =   min_border_large,                      # define top border space
                                        toolbar_location    =   None,
                                        tools               =   "")                                         # define tools to show

low_param_map_fig.xaxis.axis_label       =   x_label_map
low_param_map_fig.yaxis.axis_label       =   y_label_map
low_param_map_fig.grid.grid_line_color   =   None
low_param_map_fig.axis.minor_tick_in     =   False
low_param_map_fig.axis.minor_tick_out    =   False
low_param_map_fig.axis.major_tick_in     =   False
low_param_map_fig.outline_line_alpha     =   0
low_param_map_fig.yaxis.ticker           =   np.arange(y_range_map[0],y_range_map[1] + y_tick_map, y_tick_map)
low_param_map_fig.xaxis.ticker           =   np.arange(x_range_map[0],x_range_map[1] + x_tick_map, x_tick_map)
low_param_map_fig.background_fill_color  =   bg_color
low_param_map_fig.axis.axis_label_standoff = 10
low_param_map_fig.axis.axis_label_text_font_style = 'normal'

# stimulus circle
low_param_map_fig.circle(x = 0, y = 0, radius = stim_radius, color = stim_color)

# spans
low_param_map_fig.add_layout(Span(location = 0, dimension = 'width', line_alpha = 0.5, line_color = 'black', line_width = 1, line_dash = 'dashed'))
low_param_map_fig.add_layout(Span(location = 0, dimension = 'height', line_alpha = 0.5, line_color = 'black', line_width = 1, line_dash = 'dashed'))



# plot rf
low_param_map_fig.circle(   x                   =   deriv_model_mat[x_idx],
                            y                   =   deriv_model_mat[y_idx],
                            radius              =   deriv_model_mat[sign_idx],
                            fill_color          =   model_fill_color,
                            line_color          =   'black',
                            fill_alpha          =   1,
                            line_alpha          =   1)

time_leg_fig              =   figure(   plot_width          =   p_width,
                                        plot_height         =   int(p_height/10),
                                        x_range             =   x_range_tc,
                                        y_range             =   (0,1),
                                        x_axis_type         =   None,
                                        y_axis_type         =   None,
                                        outline_line_color  =   "white",
                                        min_border_left     =   min_border_large,
                                        min_border_right    =   min_border_large,
                                        min_border_bottom   =   min_border_large,
                                        min_border_top      =   min_border_large,
                                        toolbar_location    =   None)

stim_on = ([16,47],[48,79],[80,111],[112,143],[156,187],[188,219],[220,251],[252,283])
stim_off = ([0,15],[144,155],[284,299])
stim_dir = (['right'],['up'],['left'],['down'],['up+right'],['up+left'],['down/left'],['down+right'])

for t_stim_on in np.arange(0,len(stim_on),1):
    time_leg_fig.quad(left=stim_on[t_stim_on][0], right=stim_on[t_stim_on][1], top=1, bottom=0, fill_color="black",line_color = 'white',line_width = 1)
    x_dir_txt = (stim_on[t_stim_on][1]+stim_on[t_stim_on][0])/2.0
    time_leg_fig.text(x = x_dir_txt, y=0.35,text = stim_dir[t_stim_on],text_align = 'center',text_font_size = '6pt',text_color = 'white')

for t_stim_off in np.arange(0,len(stim_off),1):
    time_leg_fig.quad(left=stim_off[t_stim_off][0], right=stim_off[t_stim_off][1], top=1, bottom=0, fill_color="grey",line_color = 'white',line_width = 1)

# up-right space
s2 = Spacer(width=int(p_height), height=int(p_height/10))

# Put figure together
# -------------------
f                               =   column(
                                        row(time_leg_fig,s2),
                                        row(high_param_tc_fig,high_param_map_fig),
                                        row(low_param_tc_fig,low_param_map_fig))


show(f)


Loading BokehJS ...